In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('recommender').getOrCreate()

In [3]:
df = spark.read.csv('FileStore/tables/movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
df.printSchema()

root
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- userId: integer (nullable = true)

In [6]:
df.head()

Out[ 6 ]: Row(movieId=2, rating=3.0, userId=0)

In [7]:
df.show(5)

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
+-------+------+------+
only showing top 5 rows

In [8]:
df.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [9]:
df_train, df_test = df.randomSplit([0.8,0.2])

In [10]:
als = ALS(maxIter=5, regParam=0.01, 
          userCol='userId', itemCol='movieId', ratingCol='rating')

In [11]:
model = als.fit(df_train)

In [12]:
predictions = model.transform(df_test)

In [13]:
predictions.show()

+-------+------+------+-----------+
movieId|rating|userId| prediction|
+-------+------+------+-----------+
 31| 1.0| 5| 1.6656811|
 85| 1.0| 28| -1.9678504|
 85| 3.0| 1| 1.0966159|
 85| 5.0| 16| -0.8818945|
 85| 1.0| 15| -0.8210251|
 85| 5.0| 8| 1.2619215|
 85| 1.0| 29|-0.05776383|
 65| 1.0| 24| -1.4753562|
 65| 1.0| 2| 1.9171612|
 53| 3.0| 13| -0.0268578|
 53| 1.0| 6| 0.21321213|
 53| 1.0| 9| -0.5278373|
 53| 5.0| 8| -0.9899458|
 53| 1.0| 7| -0.36798|
 53| 5.0| 21|-0.35619968|
 53| 3.0| 14| -1.1428041|
 78| 1.0| 27| 0.4108026|
 78| 1.0| 12| 0.19000438|
 78| 1.0| 13| 0.96674246|
 78| 1.0| 17|-0.61651295|
+-------+------+------+-----------+
only showing top 20 rows

In [14]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [15]:
rmse = evaluator.evaluate(predictions)

In [16]:
rmse

Out[ 16 ]: 1.8732632635212574

In [17]:
single_user = df_test.filter(df_test['userId'] == 11).select(['movieId', 'userId'])

In [18]:
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 6| 11|
 11| 11|
 13| 11|
 19| 11|
 27| 11|
 35| 11|
 38| 11|
 67| 11|
 75| 11|
 78| 11|
 80| 11|
 81| 11|
 82| 11|
 88| 11|
 89| 11|
+-------+------+

In [19]:
recommendations = model.transform(single_user)

In [20]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 38| 11| 3.097983|
 88| 11| 2.568387|
 82| 11| 2.2611966|
 80| 11| 1.8474803|
 35| 11| 1.8239837|
 67| 11| 1.6783744|
 81| 11| 1.5740764|
 6| 11| 1.4404032|
 19| 11| 1.2741525|
 13| 11| 0.79994714|
 75| 11| 0.66658115|
 27| 11| 0.5110591|
 11| 11| 0.27931464|
 89| 11| 0.10154092|
 78| 11|-0.04931417|
+-------+------+-----------+